# Notebook para la clasificación de textos

## Usando GEMINI

In [ ]:
!pip install google-api-python-client
!pip install google-generativeai
!pip install google-cloud
!pip install google-cloud-vision
!pip install google
!pip install google-api-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.1/526.1 kB 11.6 MB/s eta 0:00:00


### Importación de librerías

In [ ]:
import os
import pandas as pd
import json
import re
import numpy as np

import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from sklearn.metrics import classification_report

### Configuración API de Google para generación de textos con Gemini

In [ ]:
GOOGLE_API_KEY = "TOKEN_API_GEMINI"
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

def get_completion(prompt, model="gemini-1.5-flash",history=[],api_call="completion"):
    model_gemini = genai.GenerativeModel(model)
    if api_call == "completion":
        response = model_gemini.generate_content(prompt, safety_settings=safety_settings)
    elif api_call == "chat":
        chat = model_gemini.start_chat(
            safety_settings=safety_settings,
            history=history
        )
        response = chat.send_message(prompt)
    return response.text

In [ ]:
data = pd.read_excel("Data_final.xlsx")
data_test = data[data["Dataset"]=="test"]
data_test

,ID,Texto,Categoria,Categoría de 1 a 5,Dataset
0,1,"- ¡Rápido, necesitamos sangre! - Yo soy 0 posi...",Humor,4.0,test
2,3,El otro día unas chicas llamarón a mi puerta y...,Humor,3.0,test
5,6,Van dos ciegos y le dice uno al otro: - Ojalá...,Humor,2.0,test
6,7,Noticia de última hora!! Muere una suegra at...,Humor,4.0,test
8,9,"– Mamá, mamá, ¿me haces un bocata de jamón? – ...",Humor,1.0,test
...,...,...,...,...,...
10909,10910,¿Qué país es famoso por su rueda de queso cono...,No Humor,NaN,test
10917,10918,¿Quién fue el primer hombre en pisar la luna? ...,No Humor,NaN,test
10918,10919,¿Cómo se llama la última cena de Pascua que Je...,No Humor,NaN,test
10920,10921,¿En qué país se encuentran las pirámides de Gi...,No Humor,NaN,test


# Zero Shot / Instruction based prompting

In [ ]:
data_test_p1 = data_test.copy()

In [ ]:
def get_clasification(row):
    texto = row["Texto"]
    prompt = f"""
      Tu tarea es clasificar un texto en Humor y No Humor, responde solo con la etiqueta de Humor o No Humor

      Texto: {texto}
      """
    try:
        response = get_completion(prompt)
    except:
        response = "No fue posible etiquetar"
    return response.strip()

data_test_p1['label1_gemini_INS'] = data_test_p1.apply(get_clasification, axis=1)

In [ ]:
data_test_p1

,ID,Texto,Categoria,Categoría de 1 a 5,Dataset,label1_gemini_INS
0,1,"- ¡Rápido, necesitamos sangre! - Yo soy 0 posi...",Humor,4.0,test,Humor
2,3,El otro día unas chicas llamarón a mi puerta y...,Humor,3.0,test,Humor
5,6,Van dos ciegos y le dice uno al otro: - Ojalá...,Humor,2.0,test,Humor
6,7,Noticia de última hora!! Muere una suegra at...,Humor,4.0,test,Humor
8,9,"– Mamá, mamá, ¿me haces un bocata de jamón? – ...",Humor,1.0,test,Humor
...,...,...,...,...,...,...
10909,10910,¿Qué país es famoso por su rueda de queso cono...,No Humor,NaN,test,Humor
10917,10918,¿Quién fue el primer hombre en pisar la luna? ...,No Humor,NaN,test,No Humor
10918,10919,¿Cómo se llama la última cena de Pascua que Je...,No Humor,NaN,test,Humor
10920,10921,¿En qué país se encuentran las pirámides de Gi...,No Humor,NaN,test,No Humor


In [ ]:
data_test_p2 = data_test_p1.copy()

In [ ]:
def get_clasification_1_5(row):
    texto = row["Texto"]
    prompt = f"""
      Tu tarea es clasificar el nivel de humor o gracia que puede generar un texto de humor en una escala del 1 al 5,

      Debes brindar la respuesta sin texto introductorio asignando una puntuación de 1 a 5 del nivel de gracia que genera el texto,

      Criterios:
      1 - El texto no parece humorístico o el humor es muy difícil de entender.
      2 - El humor se entiende solo si se tiene contexto cultural, geográfico o específico.
      3 - El humor es comprensible pero provoca poca gracia.
      4 - El humor es claro y genera bastante gracia.
      5 - El humor es muy claro, universalmente comprensible y provoca mucha gracia.

      Texto: {texto}
      """
    try:
        response = get_completion(prompt)
    except:
        response = "1"
    return response.strip()

data_test_p2['label2_gemini_INS'] = data_test_p2[data_test_p2["label1_gemini_INS"] == "Humor"].apply(get_clasification_1_5, axis=1)

In [ ]:
data_test_p2

,ID,Texto,Categoria,Categoría de 1 a 5,Dataset,label1_gemini_INS,label2_gemini_INS
0,1,"- ¡Rápido, necesitamos sangre! - Yo soy 0 posi...",Humor,4.0,test,Humor,4
2,3,El otro día unas chicas llamarón a mi puerta y...,Humor,3.0,test,Humor,4
5,6,Van dos ciegos y le dice uno al otro: - Ojalá...,Humor,2.0,test,Humor,4
6,7,Noticia de última hora!! Muere una suegra at...,Humor,4.0,test,Humor,4
8,9,"– Mamá, mamá, ¿me haces un bocata de jamón? – ...",Humor,1.0,test,Humor,4
...,...,...,...,...,...,...,...
10909,10910,¿Qué país es famoso por su rueda de queso cono...,No Humor,NaN,test,Humor,3
10917,10918,¿Quién fue el primer hombre en pisar la luna? ...,No Humor,NaN,test,No Humor,NaN
10918,10919,¿Cómo se llama la última cena de Pascua que Je...,No Humor,NaN,test,Humor,3
10920,10921,¿En qué país se encuentran las pirámides de Gi...,No Humor,NaN,test,No Humor,NaN


## Resultados

In [ ]:
print(classification_report(data_test_p1["Categoria"], data_test_p1['label1_gemini_INS']))

              precision    recall  f1-score   support

       Humor       0.86      1.00      0.93      1196
    No Humor       1.00      0.77      0.87       804

    accuracy                           0.91      2000
   macro avg       0.93      0.88      0.90      2000
weighted avg       0.92      0.91      0.90      2000



In [ ]:
data_humor = data_test_p2[data_test_p1["Categoria"] == "Humor"]
data_humor = data_humor[data_humor["label1_gemini_INS"] == "Humor"]
print(classification_report(data_humor["Categoría de 1 a 5"].astype(int).astype(str), data_humor['label2_gemini_INS']))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       113
           2       0.28      0.09      0.13       195
           3       0.39      0.14      0.21       434
           4       0.36      0.84      0.51       407
           5       0.13      0.09      0.10        46

    accuracy                           0.36      1195
   macro avg       0.23      0.23      0.19      1195
weighted avg       0.32      0.36      0.27      1195



# Few-Shot Prompting

In [ ]:
def get_clasification(row):
    texto = row["Texto"]
    prompt = f"""
      Tu tarea es clasificar un texto en 'Humor' o 'No Humor'. Responde solo con una de las dos etiquetas, sin texto adicional.

      Ejemplos:
      Texto: ¿Cuál es el animal más antiguo? – La vaca – ¿Por qué? – Porque está en blanco y negro.
      Etiqueta: Humor

      Texto: Hoy se cumplen 10 años desde que empecé mi trabajo actual. Ha sido un camino lleno de aprendizajes y desafíos.
      Etiqueta: No Humor

      Texto: {texto}
      Etiqueta:"""
    try:
        response = get_completion(prompt)
    except:
        response = "No fue posible etiquetar"
    return response.strip()

data_test_p1['label1_gemini_FS'] = data_test_p1.apply(get_clasification, axis=1)

In [ ]:
def get_clasification_1_5(row):
    texto = row["Texto"]
    prompt = f"""
      Tu tarea es clasificar el nivel de humor o gracia que puede generar un texto de humor en una escala del 1 al 5,

      Debes brindar la respuesta sin texto introductorio asignando una puntuación de 1 a 5 del nivel de gracia que genera el texto,

      Ejemplos:
      Texto: ¿Cuál es el animal más antiguo?  - La vaca - ¿Por qué?  - Porqué esta en blanco y negro.
      Categoría: 5

      Texto: En qué se convierte un país en el que se prohíben los tacos? En un país destacado!
      Categoría: 4

      Texto: Mamá, mamá, en el colegio dicen que estoy loco. – ¿Y quién dice eso de ti? – ...Me lo dicen las sillas...
      Categoría: 3

      Texto: No tienes ni idea de lo que soporto - Una ciudad de Portugal
      Categoría: 2

      Texto: ¿Oiste hablar de ese experimento que hicieron para ver si trabajar con computadores es malo para la salud? Metieron a tres ratas dentro de una jaula al lado de un computador, y lo dejaron encendido durante dos meses.
      Categoría: 1

      Texto: {texto}
      """
    try:
        response = get_completion(prompt)
    except:
        response = "1"
    return response.strip()

data_test_p1['label2_gemini_FS'] = data_test_p1[data_test_p1["label1_gemini_FS"] == "Humor"].apply(get_clasification_1_5, axis=1)

## Resultados

In [ ]:
print(classification_report(data_test_p1["Categoria"], data_test_p1['label1_gemini_FS']))

              precision    recall  f1-score   support

       Humor       0.94      1.00      0.97      1196
    No Humor       0.99      0.91      0.95       804

    accuracy                           0.96      2000
   macro avg       0.97      0.95      0.96      2000
weighted avg       0.96      0.96      0.96      2000



In [ ]:
data_humor = data_test_p1[data_test_p1["Categoria"] == "Humor"]
data_humor = data_humor[data_humor["label1_gemini_FS"] == "Humor"]
print(classification_report(data_humor["Categoría de 1 a 5"].astype(int).astype(str), data_humor['label2_gemini_FS']))

              precision    recall  f1-score   support

           1       0.14      0.08      0.10       111
           2       0.23      0.08      0.12       195
           3       0.38      0.08      0.13       433
           4       0.37      0.41      0.39       406
           5       0.05      0.52      0.08        46

    accuracy                           0.21      1191
   macro avg       0.23      0.23      0.17      1191
weighted avg       0.32      0.21      0.21      1191



# CoT

In [ ]:
def get_clasification(row):
    texto = row["Texto"]
    prompt = f"""
    Tu tarea es clasificar un texto como Humor o No Humor.

    Piensa paso a paso para decidir si el texto tiene elementos humorísticos (como ironía, juegos de palabras, exageraciones o situaciones absurdas), pero no muestres tu razonamiento. Al final, responde únicamente con una de estas dos etiquetas: Humor o No Humor.

    Texto: {texto}

    Respuesta:
    """
    try:
        response = get_completion(prompt)
    except:
        response = "No fue posible etiquetar"
    return response.strip()

data_test_p1['label1_gemini_CoT'] = data_test_p1.apply(get_clasification, axis=1)

In [ ]:
def get_clasification_1_5(row):
    texto = row["Texto"]
    prompt = f"""
    Tu tarea es clasificar el nivel de humor o gracia que puede generar un texto en una escala del 1 al 5.

    Piensa paso a paso internamente para decidir qué tan gracioso es el texto, según los siguientes criterios. No muestres tu razonamiento. Al final, responde únicamente con un número del 1 al 5, sin ningún texto adicional.

    Criterios:
    1 - El texto no parece humorístico o el humor es muy difícil de entender.
    2 - El humor se entiende solo si se tiene contexto cultural, geográfico o específico.
    3 - El humor es comprensible pero provoca poca gracia.
    4 - El humor es claro y genera bastante gracia.
    5 - El humor es muy claro, universalmente comprensible y provoca mucha gracia.

    Texto: {texto}

    Respuesta:
    """
    try:
        response = get_completion(prompt)
    except:
        response = "1"
    return response.strip()

data_test_p1['label2_gemini_CoT'] = data_test_p1[data_test_p1["label1_gemini_CoT"] == "Humor"].apply(get_clasification_1_5, axis=1)

## Resultados

In [ ]:
print(classification_report(data_test_p1["Categoria"], data_test_p1['label1_gemini_CoT']))

              precision    recall  f1-score   support

       Humor       0.86      0.99      0.92      1196
    No Humor       0.99      0.76      0.86       804

    accuracy                           0.90      2000
   macro avg       0.93      0.88      0.89      2000
weighted avg       0.91      0.90      0.90      2000



In [ ]:
data_humor = data_test_p1[data_test_p1["Categoria"] == "Humor"]
data_humor = data_humor[data_humor["label1_gemini_CoT"] == "Humor"]
print(classification_report(data_humor["Categoría de 1 a 5"].astype(int).astype(str), data_humor['label2_gemini_CoT']))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       112
           2       0.18      0.01      0.02       194
           3       0.40      0.09      0.15       433
           4       0.36      0.93      0.51       405
           5       0.19      0.07      0.10        46

    accuracy                           0.35      1190
   macro avg       0.23      0.22      0.16      1190
weighted avg       0.30      0.35      0.24      1190



# Se asegura de que el dataframe final tenga todas las columnas y se guardan los resultados

In [ ]:
data_test_final = data_test_p1.copy()
data_test_final['label2_gemini_INS'] = data_test_p2['label2_gemini_INS']
data_test_final

,ID,Texto,Categoria,Categoría de 1 a 5,Dataset,label1_gemini_INS,label1_gemini_FS,label2_gemini_FS,label1_gemini_CoT,label2_gemini_CoT,label2_gemini_INS
0,1,"- ¡Rápido, necesitamos sangre! - Yo soy 0 posi...",Humor,4.0,test,Humor,Humor,2,Humor,4,4
2,3,El otro día unas chicas llamarón a mi puerta y...,Humor,3.0,test,Humor,Humor,3,Humor,4,4
5,6,Van dos ciegos y le dice uno al otro: - Ojalá...,Humor,2.0,test,Humor,Humor,5,Humor,4,4
6,7,Noticia de última hora!! Muere una suegra at...,Humor,4.0,test,Humor,Humor,1,Humor,4,4
8,9,"– Mamá, mamá, ¿me haces un bocata de jamón? – ...",Humor,1.0,test,Humor,Humor,3,Humor,4,4
...,...,...,...,...,...,...,...,...,...,...,...
10909,10910,¿Qué país es famoso por su rueda de queso cono...,No Humor,NaN,test,Humor,No Humor,NaN,Humor,3,3
10917,10918,¿Quién fue el primer hombre en pisar la luna? ...,No Humor,NaN,test,No Humor,No Humor,NaN,No Humor,NaN,NaN
10918,10919,¿Cómo se llama la última cena de Pascua que Je...,No Humor,NaN,test,Humor,Humor,1,Humor,3,3
10920,10921,¿En qué país se encuentran las pirámides de Gi...,No Humor,NaN,test,No Humor,No Humor,NaN,No Humor,NaN,NaN


In [ ]:
data_test_final = data_test_final.drop("Texto", axis=1)
data_test_final.to_csv('data_test_final.csv', index=False)